导入程序所需要的包

In [31]:
import torch
import torch.utils.data as DataSet
import torch.nn as nn
import torch.optim
from torch.autograd import Variable
import torch.nn.functional as F
# from torchsummary import summary
import string
import numpy as np
import time

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
device

device(type='cpu')

读入，进行数据预处理并展示数据

In [32]:
f = open(r'C:\Users\houyu\pythonProject\RNN文本生成\poems_clean.txt', encoding='utf-8')
poems = []
for line in f.readlines():
    title, poem = line.split(':')
    poem = poem.replace(' ', '')
    poem = poem.replace('\n', '')
    if len(poem) > 0 :
        poems.append(list(poem))

print(poems[0][:])

['寒', '随', '穷', '律', '变', '春', '逐', '鸟', '声', '开', '初', '风', '飘', '带', '柳', '晚', '雪', '间', '花', '梅', '碧', '林', '青', '旧', '竹', '绿', '沼', '翠', '新', '苔', '芝', '田', '初', '雁', '去', '绮', '树', '巧', '莺', '来']


创建字符编码字典

In [33]:
word2idx = {}
i = 1
for poem in poems:
    for word in poem:
        if word2idx.get(word) == None:
            word2idx[word] = i
            i += 1
word2idxl[10]

{'寒': 1,
 '随': 2,
 '穷': 3,
 '律': 4,
 '变': 5,
 '春': 6,
 '逐': 7,
 '鸟': 8,
 '声': 9,
 '开': 10,
 ...}

In [34]:
len(word2idx)

5545

对诗歌进行编码，从原始数据到矩阵

In [35]:
poems_digit = []
for poem in poems:
    poem_digit = []
    for word in poem:
        poem_digit.append(word2idx[word])
    poems_digit.append(poem_digit)
    
print("原始诗歌")
print(poems[3829])
print("\n 编码后的结果")
print(poems_digit[3829][:])

原始诗歌
['春', '眠', '不', '觉', '晓', '处', '处', '闻', '啼', '鸟', '夜', '来', '风', '雨', '声', '花', '落', '知', '多', '少']

 编码后的结果
[6, 2420, 57, 2468, 451, 198, 198, 747, 376, 8, 228, 39, 12, 270, 9, 19, 319, 67, 510, 1941]


拆分X、Y变量并处理长短不一问题

In [36]:
# 设置诗歌最大长度为50个字符
maxlen = 50
X = []
Y = []
for poem_digit in poems_digit:
    y=poem_digit[1:]+[0]*(maxlen - len(poem_digit))
    Y.append(y)
    # 将最后一个字符之前的部分作为X，并补齐字符
    x = poem_digit[:-1] + [0]*(maxlen - len(poem_digit))
    X.append(x)
    
print("原始诗歌")
print(poems[3829])
print("变量X")
print(X[3829])
print("变量Y")
print(Y[3829])

原始诗歌
['春', '眠', '不', '觉', '晓', '处', '处', '闻', '啼', '鸟', '夜', '来', '风', '雨', '声', '花', '落', '知', '多', '少']
变量X
[6, 2420, 57, 2468, 451, 198, 198, 747, 376, 8, 228, 39, 12, 270, 9, 19, 319, 67, 510, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
变量Y
[2420, 57, 2468, 451, 198, 198, 747, 376, 8, 228, 39, 12, 270, 9, 19, 319, 67, 510, 1941, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


划分训练集和测试集，将所有数据的顺序打乱重排

In [37]:
idx = np.random.permutation(range(len(X)))
X = [X[i] for i in idx]
Y = [Y[i] for i in idx]

# 切分出1/5的数据放入校验集    
validX = X[:len(X) // 5]
trainX = X[len(X) // 5:]
validY = Y[:len(Y) // 5]
trainY = Y[len(Y) // 5:]

In [38]:
batch_size = 64
train_ds = DataSet.TensorDataset(torch.IntTensor(np.array(trainX, dtype=int)), torch.IntTensor(np.array(trainY, dtype=int)))
train_loader = DataSet.DataLoader(train_ds, batch_size=batch_size, shuffle=True, num_workers=4)

valid_ds = DataSet.TensorDataset(torch.IntTensor(np.array(validX, dtype=int)), torch.IntTensor(np.array(validY, dtype=int)))
valid_loader = DataSet.DataLoader(valid_ds, batch_size=batch_size, shuffle=False, num_workers=4)

In [39]:
class SimpleRNN(nn.Module):
    def __init__(self, output_size, word_num, embedding_size, hidden_size, num_layers=1):
        super(SimpleRNN, self).__init__()
        self.embedding = nn.Embedding(word_num, embedding_size)

        self.rnn = nn.RNN(embedding_size, hidden_size, num_layers, batch_first=True)

        self.fc = nn.Linear(hidden_size, output_size)
                
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        self.output_size = output_size
        
    def forward(self, x, hidden):
        # 运算过程:先进行embedding层的计算
        x = self.embedding(x)

        # x的尺寸为：batch_size，num_step，hidden_size
        output, hidden = self.rnn(x, hidden)
        # output的尺寸为：batch_size，maxlen-1, hidden_size
        output = self.fc(output)
        output = output.view(-1,output.shape[-1])  #为便于后续处理，此处进行展平
        # output的尺寸为：batch_size*(maxlen-1)，output_size
        return output, hidden
    
    def initHidden(self, batch_size):
        # 对隐含单元初始化
        # 尺寸是layer_size，batch_size，hidden_size
        return Variable(torch.zeros(self.num_layers, batch_size, self.hidden_size))

In [40]:
vocab_size = len(word2idx.keys()) + 1  # 获取文本数据集中包含的字符数量

lr = 1e-3
epochs = 50

# 生成一个简单的RNN，输入size为49（50-1），输出size为vocab_size（字符总数）
rnn = SimpleRNN(output_size=vocab_size, word_num=vocab_size, embedding_size=64, hidden_size=128)
rnn = rnn.to(device)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(rnn.parameters(), lr=lr)

rnn

SimpleRNN(
  (embedding): Embedding(5546, 64)
  (rnn): RNN(64, 128, batch_first=True)
  (fc): Linear(in_features=128, out_features=5546, bias=True)
)


In [41]:
'''
计算预测错误率的函数，pre是模型给出的一组预测结果（batch_size行、num_classes列的矩阵），label是正确标签
'''
def accuracy(pre, label):
    pre = torch.max(pre.data, 1)[1] # 得到每一行（每一个样本）输出值最大元素的下标
    rights = pre.eq(label.data).sum()
    acc = rights.data/len(label)
    return acc.float()

模型验证

In [42]:
def validate(model, val_loader):
    # 在校验集上运行一遍并计算损失和准确率
    val_loss = 0
    val_acc = 0
    model.eval()
    for batch, data in enumerate(val_loader):
        init_hidden = model.initHidden(len(data[0]))
        init_hidden = init_hidden.to(device)
        x, y = Variable(data[0]), Variable(data[1])
        x, y = x.to(device),y.to(device)
        outputs, hidden = model(x, init_hidden)
        y = y.long()
        y = y.view(y.shape[0]*y.shape[1])
        loss = criterion(outputs, y)
        val_loss += loss.data.cpu().numpy()  
        val_acc += accuracy(outputs, y)
    val_loss /= len(val_loader)
    val_acc /= len(val_loader)
    return val_loss, val_acc

In [43]:
# 打印训练结果
def print_log(epoch, train_time, train_loss, train_acc, val_loss, val_acc, epochs=10):
    print(f"Epoch [{epoch}/{epochs}], time: {train_time:.2f}s, loss: {train_loss:.4f}, acc: {train_acc:.4f}, val_loss: {val_loss:.4f}, val_acc: {val_acc:.4f}")

# 定义主函数：模型训练

In [44]:
def train(model,optimizer, train_loader, val_loader, epochs=1):
    train_losses = []
    train_accs = []
    val_losses = []
    val_accs = []
    
    for epoch in range(epochs):
        train_loss = 0
        train_acc = 0
        start = time.time()
        for batch, data in enumerate(train_loader):
            model.train() # 标志当前RNN处于训练阶段
            init_hidden = model.initHidden(len(data[0])) # 初始化隐含层单元
            init_hidden = init_hidden.to(device)
            optimizer.zero_grad()
            x, y = Variable(data[0]), Variable(data[1])
            x, y = x.to(device), y.to(device)
            outputs, hidden = model(x, init_hidden) # 输入RNN，产生输出
            y = y.long()
            y = y.view(y.shape[0]*y.shape[1])
            loss = criterion(outputs, y)
            train_loss += loss.data.cpu().numpy()
            train_acc += accuracy(outputs, y)
            loss.backward()
            optimizer.step()
        
        end = time.time()
        train_time = end - start
        train_loss /= len(train_loader)
        train_acc /= len(train_loader)
        val_loss, val_acc = validate(model, val_loader)
        train_losses.append(train_loss)
        train_accs.append(train_acc)
        val_losses.append(val_loss)
        val_accs.append(val_acc) 
        print_log(epoch + 1, train_time, train_loss, train_acc, val_loss, val_acc, epochs=epochs)
    return train_losses, train_accs, val_losses, val_accs

# 模型训练

In [69]:
history = train(rnn, optimizer, train_loader, valid_loader, epochs=epochs)

Epoch [1/50], time: 32.31s, loss: 4.2088, acc: 0.3779, val_loss: 4.2160, val_acc: 0.3836
Epoch [2/50], time: 31.79s, loss: 4.1232, acc: 0.3823, val_loss: 4.1594, val_acc: 0.3865
Epoch [3/50], time: 30.19s, loss: 4.0463, acc: 0.3857, val_loss: 4.1088, val_acc: 0.3893
Epoch [4/50], time: 31.14s, loss: 3.9778, acc: 0.3888, val_loss: 4.0680, val_acc: 0.3916
Epoch [5/50], time: 29.85s, loss: 3.9206, acc: 0.3916, val_loss: 4.0384, val_acc: 0.3937
Epoch [6/50], time: 53.65s, loss: 3.8711, acc: 0.3947, val_loss: 4.0107, val_acc: 0.3957
Epoch [7/50], time: 29.67s, loss: 3.8265, acc: 0.3971, val_loss: 3.9876, val_acc: 0.3974
Epoch [8/50], time: 28.65s, loss: 3.7849, acc: 0.3999, val_loss: 3.9726, val_acc: 0.3989
Epoch [9/50], time: 26.52s, loss: 3.7505, acc: 0.4018, val_loss: 3.9531, val_acc: 0.4008
Epoch [10/50], time: 27.59s, loss: 3.7192, acc: 0.4035, val_loss: 3.9407, val_acc: 0.4015
Epoch [11/50], time: 26.64s, loss: 3.6904, acc: 0.4053, val_loss: 3.9310, val_acc: 0.4022
Epoch [12/50], time

额外项目：用RNN写藏头诗

In [67]:
poem_incomplete = '侯****大****帅****哥****'
poem_index = [] # 用于记录诗歌创作过程中字符和整数的对应关系
poem_text = '' # 记录诗歌的创作过程，循环结束后应是一首完整的诗

init_hidden = rnn.initHidden(1)
init_hidden = init_hidden.to(device)
for i in range(len(poem_incomplete)):
    current_word = poem_incomplete[i]
    if current_word != '*':# 若当前的字符不是"*"，使用word2idx字典将其变为一个整数
        index = word2idx[current_word]
    else:        # 若当前的字符是"*"，需要用RNN模型对其进行预测
        x = poem_index + [0]*(maxlen -1 - len(poem_index))
        x = torch.IntTensor(np.array([x], dtype=int))
        x = Variable(x)
        x = x.to(device)
        pre, hidden = rnn(x, init_hidden) 
        init_hidden = hidden.data
        crt_pre = pre[i-1].cpu() # 获取第i个时间步的输出，对应位置i-1
        index = torch.argmax(crt_pre)
        current_word = [k for k, v in word2idx.items() if v == index][0]
    poem_index.append(index) 
    poem_text = poem_text + current_word

In [68]:
print(poem_text[0:5])
print(poem_text[5:10])
print(poem_text[10:15])
print(poem_text[15:20])

侯门前路远
大日暮雨雪
帅人间不知
哥水上山中
